<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Estadistica_Piez%C3%B3metros_El%C3%A9ctricos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🔍 FASE 1 — Estadística Descriptiva: Piezómetros Eléctricos

##🎯 Objetivo de esta etapa:
Obtener una comprensión clara de las variables medidas por los piezómetros eléctricos a través de estadísticas descriptivas y visualizaciones.

In [1]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={'PE175_20250608.csv': {'metadata': {'name': 'PE175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: PIEZOMETROS_ELECTRICOS (XLSX)


,FECHA,PROGRESIVA,PIEZOMETRO,COTA_RIO_(M.S.N.M),LECTURA_CUERDA_VIBRANTE,TEMPERATURA_(°C),MCA_1_(FACTOR_G),MCA_2_(FACTOR_G_Y_K),COTA_NF,PRECIPITACIONES_(MM)
0,18/07/2023,0+175,PE 7,115.156,8938.93,2.7,0.030356,0.054266,119.836356,NaN
1,29/07/2023,0+175,PE 7,114.831,8911.57,0.9,0.806559,0.859161,120.612559,NaN
2,15/08/2023,0+175,PE 7,114.629,8939.79,1.5,0.005958,0.048996,119.811958,NaN
3,24/08/2023,0+175,PE 7,114.391,8949.63,1.5,-0.273203,-0.230165,SIN NF,3.8
4,01/09/2023,0+175,PE 7,114.309,8953.65,1.6,-0.387250,-0.345806,SIN NF,NaN


In [2]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Diccionario con datos cargados
# datos_csv["piezometros_electricos"]
# datos_xlsx["piezometros_electricos"]

# --- Selectores ---
origen_selector = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:"
)

progresiva_selector = widgets.Dropdown(
    options=[],
    description="Progresiva:"
)

piezometro_selector = widgets.Dropdown(
    options=[],
    description="Piezómetro:"
)

boton_mostrar = widgets.Button(description="Mostrar datos", button_style='success')
salida = widgets.Output()

# --- Funciones auxiliares ---

def actualizar_progresivas(change=None):
    origen = origen_selector.value
    df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]

    if df.empty:
        progresiva_selector.options = []
        return

    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresivas = sorted(df['PROGRESIVA'].dropna().unique())
    progresiva_selector.options = progresivas
    if progresivas:
        progresiva_selector.value = progresivas[0]

def actualizar_piezometros(change=None):
    origen = origen_selector.value
    df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]

    if df.empty or progresiva_selector.value is None:
        piezometro_selector.options = []
        return

    df_filtrado = df[df["PROGRESIVA"] == progresiva_selector.value]
    piezometros = sorted(df_filtrado["PIEZOMETRO"].dropna().unique())
    piezometro_selector.options = piezometros
    if piezometros:
        piezometro_selector.value = piezometros[0]

def mostrar_datos(b):
    with salida:
        clear_output()
        origen = origen_selector.value
        df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]

        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        df = df[df["PROGRESIVA"] == progresiva_selector.value]
        df = df[df["PIEZOMETRO"] == piezometro_selector.value]

        print(f"📌 Datos seleccionados: Origen = {origen}, Progresiva = {progresiva_selector.value}, Piezómetro = {piezometro_selector.value}")
        display(df.head(10))

# --- Conexión de eventos ---
origen_selector.observe(actualizar_progresivas, names="value")
progresiva_selector.observe(actualizar_piezometros, names="value")
boton_mostrar.on_click(mostrar_datos)

# --- Mostrar interfaz ---
display(widgets.HBox([origen_selector, progresiva_selector, piezometro_selector, boton_mostrar]))
display(salida)

# Inicialización
actualizar_progresivas()
actualizar_piezometros()


Output()